In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
# Parameters
INPUT_FILE = './data/champeng_4_pair_rank.csv'
MAX_ORDER = 10
SCORE_LIST = [34,21,13,8,5,3,2,1,1,1]

assert len(SCORE_LIST) == MAX_ORDER

In [3]:
def clean_rank_score(text):
    """ Get the pair number from each cell of rankings
        If the cell is blank, return nan
    """
    try:
        return 'pair ' + re.search(r"\d+", text).group(0)
    except:
        return np.nan
    
def apply_to_table(func, df, name_condition, mode = 'in'):
    """ Apply a function to every column of a dataset which matches the given condition
        
        If mode = 'in' (default),
        the condition is a column's name must contain name_condition
        
        If mode = 'equal',
        the condition is a column's name must equal to name_condition
        
        If mode = 'not in',
        the condition is a column's name must not contain name_condition
        
        If mode = 'not equal',
        the condition is a column's name must not equal to name_condition
    """
    new = df.copy()
    
    if mode == 'in':
        selected_cols = [col for col in df if name_condition in col]
    elif mode == 'equal':
        selected_cols = [col for col in df if col == name_condition]
    elif mode == 'not in':
        selected_cols = [col for col in df if name_condition not in col]
    elif mode == 'not equal':
        selected_cols = [col for col in df if col != name_condition]
    else:
        return "Error: mode is not defined"
    
    for col in new.columns:
        if col in selected_cols:
            new[col] = df[col].apply(func)
    return new

In [4]:
data = pd.read_csv(INPUT_FILE)

In [5]:
new_col_names = ['timestamp', 'name', 'ID', 'year', 'major']
new_col_names.extend([''.join(['rank', str(i + 1)]) for i in range(MAX_ORDER)])
data.columns = new_col_names

# Remove duplicates (keep last)
data.drop_duplicates(subset = 'ID', keep = 'last', inplace = True)
# new_df = data.drop_duplicates(subset='ID', keep='last')

# Edit the one of which ID is wrong (60317499221 -> 6031749221)
# data.replace({60317499221: 6031749221}, inplace = True)

# Set index
data.set_index('ID', inplace = True)

In [6]:
# data.shape

In [7]:
# Drop unused columns
data.drop(['timestamp', 'major', 'year', 'name'], axis = 1, inplace = True)

In [8]:
ranks = data[[col for col in data if 'rank' in col]]
#ranks.head(3)

In [9]:
rank_cleaned = apply_to_table(clean_rank_score, ranks, 'rank', 'in')

## Flag Check: Select Duplicate Pairs

If a candidate select the same pair in a different rank, we will use only the lower rank and will replace the higher rank with some random pair that is not selected much.

In [12]:
def dup_flag_old(ranks):
    unique_list = []
    dup_list = []
    for pair in ranks:
        if pair not in unique_list:
            unique_list.append(pair)
        else:
            dup_list.append(pair)
    # return '-'.join(dup_list)
    if len(dup_list) == 0:
        return 0
    else:
        return 1 

def dup_flag(ranks):
    if ranks.nunique() == MAX_ORDER:
        return 0
    else: return 1


In [14]:
rank_cleaned['dup_flag'] = rank_cleaned.apply(dup_flag, axis = 1)
rank_cleaned.head(3)

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,dup_flag
ID,,,,,,,,,,,
6338110221,pair 16,pair 18,pair 8,pair 13,pair 17,pair 6,pair 5,pair 4,pair 2,pair 3,0
6030127721,pair 7,pair 17,pair 14,pair 15,pair 6,pair 16,pair 8,pair 12,pair 9,pair 13,0
6130584921,pair 2,pair 3,pair 15,pair 16,pair 17,pair 7,pair 10,pair 11,pair 12,pair 14,0


In [18]:
# assert rank_cleaned['dup_flag'].sum() == 0
# rank_cleaned['dup_flag'].sum()
rank_cleaned[rank_cleaned['dup_flag'] > 0]

,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,rank10,dup_flag
ID,,,,,,,,,,,
6230213721,pair 4,pair 7,pair 13,pair 12,pair 16,pair 7,pair 17,pair 15,pair 14,pair 10,1
6230529021,pair 7,pair 5,pair 14,pair 12,pair 16,pair 4,pair 11,pair 8,pair 16,pair 17,1
6230008421,pair 3,pair 2,pair 12,pair 13,pair 12,pair 5,pair 4,pair 1,pair 16,pair 7,1


If there is no duplicate, move on to the next step.

# Create a New Table

In [50]:
col_list = [' '.join(['pair', str(i)]) for i in range(1, 19)]

# Drop duplicate flag from rank_cleaned
rank_cleaned.drop(['dup_flag'], axis = 1, inplace = True)

scores = pd.DataFrame(index=rank_cleaned.index, columns=col_list)

In [51]:
for candidate_id in rank_cleaned.index:
    row = rank_cleaned.loc[candidate_id]
    for rank in row.index:
        # Remove the word "rank"
        rank_number = rank[4:]
        
        scores.loc[candidate_id, row.loc[rank]] = rank_number
        
# Reorder columns
pair_order = [' '.join(['pair', str(i)]) for i in range(1, 19)]
scores = scores[pair_order]

In [52]:
scores

,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18
ID,,,,,,,,,,,,,,,,,,
6338110221,NaN,9,10,8,7,6,NaN,3,NaN,NaN,NaN,NaN,4,NaN,NaN,1,5,2
6030127721,NaN,NaN,NaN,NaN,NaN,5,1,7,9,NaN,NaN,8,10,3,4,6,2,NaN
6130584921,NaN,1,2,NaN,NaN,NaN,6,NaN,NaN,7,8,9,NaN,10,3,4,5,NaN
6238084521,NaN,NaN,NaN,NaN,4,1,NaN,2,5,NaN,7,9,NaN,NaN,6,3,10,8
6230078321,NaN,NaN,NaN,NaN,10,6,9,8,NaN,NaN,1,4,5,NaN,3,7,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230274021,NaN,NaN,NaN,NaN,10,9,3,NaN,NaN,NaN,4,5,6,NaN,2,7,1,8
6130131221,NaN,9,7,NaN,NaN,3,10,NaN,NaN,4,5,8,NaN,NaN,1,2,NaN,6
6330068121,NaN,2,1,NaN,NaN,NaN,9,NaN,NaN,4,NaN,7,10,8,NaN,6,3,5


In [53]:
# Save
scores.to_csv('./res/pair_selection.csv')

In [56]:
data.rank1.value_counts().sort_values(ascending = False)
# data

Pair 7. ดร. วิบูลย์ อรุณธเนศ (พี่เห่า) และ คุณแกมแพร ศุภกรโกศัย (พี่แกม)                18
Pair 15. คุณรัชดา ด่านพงษ์เจริญ (พี่อ้อ) และ คุณวิธวินท์ อิทธิภาณุวัต (พี่วิน)          17
Pair 8. คุณสันติ ศรีวิชาญกุล (พี่ตั้ม) และ คุณวิโรจน์ จิรพัฒนกุล (พี่ต้า)               13
Pair 11. คุณ อโณทัย อดุลพันธุ์(พี่โน) และ คุณพชร อารายะการกูล (พี่โบ๊ท)                  9
Pair 17. คุณพิเชษฐ สิทธิอำนวย (พี่โบ้) และ คุณถิรนันท์ อรุณวัฒนกูล (พี่หนึ่ง)            7
Pair 6. คุณศิริวัฒน์ วงศ์จารุกร (พี่เล้ง) และ คุณสรวิศ ไพบูลย์รัตนากร (พี่ยีราฟ)         7
Pair 16. คุณสาธร อุพันวัน (พี่เต๋อ) และ คุณกิตติศักดิ์ ปัญญาจิรกุล (พี่หวัง)             6
Pair 12. ปริมล กาญจนจารี (พี่ใหม่) และ คุณโทนี่ คันธาภัสระ (พี่โทนี่)                    6
Pair 2. คุณวิศรุต ศรีโรจนกุล (พี่ต๋อม) และ คุณธนวัฒน์ ว่องเศรษฐชัย (พี่เอ็ม)             5
Pair 13. คุณเสนธิป ศรีไพพรรณ (พี่เส) และ คุณวีรวัฒน์ คิรินทร์รัตนะ (พี่วี)               5
Pair 4. คุณวิษณุ วิวัฒนะประเสริฐ (พี่เหมา) และ ดร. ธิติ วัชรสินธพชัย (พี่เก่ง)           4

## Add Tier

In [17]:
app_score = pd.read_excel('Data/application_score.xlsx', sheet_name='final', index_col='ID')

In [18]:
score_with_tier = scores.copy()

In [19]:
score_with_tier['tier'] = app_score['Status']
score_with_tier.head()

,pair 1,pair 2,pair 3,pair 4,pair 5,pair 6,pair 7,pair 8,pair 9,pair 10,pair 11,pair 12,pair 13,pair 14,pair 15,pair 16,pair 17,pair 18,tier
ID,,,,,,,,,,,,,,,,,,,
6230124521,3,2,NaN,NaN,4,10,NaN,9,NaN,7,NaN,8,NaN,NaN,5,NaN,1,6,4
6031843521,NaN,NaN,3,NaN,NaN,8,5,2,NaN,6,NaN,9,7,NaN,NaN,1,4,10,2
6030274221,NaN,6,1,NaN,7,9,NaN,5,NaN,10,NaN,3,NaN,NaN,2,NaN,8,4,4
6031779021,4,NaN,2,NaN,NaN,NaN,NaN,1,5,6,NaN,3,NaN,NaN,7,8,10,9,4
6030826721,10,1,4,NaN,NaN,NaN,NaN,5,NaN,NaN,9,6,2,NaN,8,NaN,3,7,1


In [23]:
score_with_tier.to_csv('pair_selection_with_tier.csv')

In [25]:
score_with_tier.tier.value_counts()

4    41
2    29
3    27
1    11
Name: tier, dtype: int64